## Add the Fashion MNIST model to the Flux model-zoo ##

1. Write Flux model
2. Benchmark it against Pytorch, Tensorflow equivalents

### 1. Import Libraries ###
Note: Due to compatability issues on my laptop, I cannot properly install CUDA (hence Cudnn) and cannot load the arrays to my GPU :(

In [1]:
using MLDatasets
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON

### 2. Data Preparation ###
I load the train and test sets of FashionMNIST to accessible formats, and define a function to partition data into mini-batches.

In [2]:
train_x, train_y = FashionMNIST.traindata()
test_x, test_y = FashionMNIST.testdata()
train_x, test_x = [train_x[:, :, i] for i in 1:size(train_x,3)], [test_x[:, :, i] for i in 1:size(test_x,3)]


# Mini-batch function:
function mb(img, label, indexx) 
    mb_x = Array{Float32}(undef, size(img[1])..., 1, length(indexx)) # placeholder
    for i in 1:length(indexx)
        mb_x[:, :, :, i] = Float32.(img[indexx[i]]) # Asigns images into batches of 64
    end
    # This encodes labels into one-hot vector representation
    mb_y = onehotbatch(label[indexx], 0:9)
    return (mb_x, mb_y)
end

mb (generic function with 1 method)

### 3. Define the Model ###
I construct my ConvNet and define Mean Squared Error as the cost function.

In [3]:
ConvNet = Chain(
    # Input: (28, 28, 1, 64)
    
    # After Conv1: (14, 14, 16, 64)
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),

    # After Conv2: (7, 7, 32, 64)
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),

    # After Conv3: (3, 3, 64, 64)
    Conv((3, 3), 32=>64, pad=(1,1), relu),
    MaxPool((2,2), stride=(2,2)),

    # Flatten to (64, 576)
    x -> reshape(x, :, size(x, 4)),
    Dense(576, 72),
    x -> relu.(x),
    Dense(72, 10),
    x -> relu.(x),
    
    # Get probability predictions, corresponding to each class
    softmax,
)

# MSE Loss
function loss(x, y)
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x)) 
    # Augmenting x helps increase inter-class variance (decision boundary) >> improves model discriminativeness
    y_hat = ConvNet(x_aug)
    return crossentropy(y_hat, y)
end

loss (generic function with 1 method)

### 4. Define Hyperparameters ###
I set the mini-batch size as 64, learning-rate as 0.001, epoch number as 200; and use the Adam optimiser.

In [4]:
mb_size = 64
mb_idxs = partition(1:length(train_x), mb_size)
trainset = [mb(train_x, train_y, i) for i in mb_idxs]
testset = mb(test_x, test_y, 1:length(test_x))
deleteat!(trainset, 938) # dims error

# pass data to GPU for acceleration
#trainset, testset, ConvNet = gpu.(trainset), gpu.(testset), gpu(ConvNet)

accuracy(x, y) = mean(onecold(ConvNet(x)) .== onecold(y))
optimiser = ADAM(0.001)
best_acc, last_improvement, epoch_num, threshold = 0.0, 0, 100, 0.95

(0.0, 0, 100, 0.95)

In [5]:
# Sanity Check: (classes=10, minibatch=64) array
ConvNet(trainset[1][1])

10×64 Array{Float32,2}:
 0.105307   0.108713   0.102776   …  0.108729   0.102043   0.10187  
 0.107922   0.101713   0.100319      0.104436   0.109957   0.10227  
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966
 0.0980129  0.0986968  0.0995453  …  0.0983543  0.101364   0.0996767
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966
 0.100681   0.0986968  0.100088      0.0983543  0.100523   0.100404 
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966
 0.0980129  0.0986968  0.0995453     0.0983543  0.0976855  0.0992966

### 5. Begin Training & Testing ###

In [6]:
time = 0.0

for epoch in 1:epoch_num
    global best_acc, last_improvement
    
    tic = time_ns()
    # Train for a single epoch
    Flux.train!(loss, params(ConvNet), trainset, optimiser)
    toc = time_ns()
    time += toc-tic
    println(time/1.0e9)
    # Calculate accuracy:
    acc = accuracy(testset...)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))

    # Early exit at 95% accuracy
    if acc >= threshold
        @info(" -> Early-exiting: We reached our target accuracy of 90.0%")
        break
    end

    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to fmnist_flux.bson")
        BSON.@save joinpath("fmnist_flux_4.bson") ConvNet epoch acc
        best_acc, last_improvement = acc, epoch
    end

    # Learning rate decay to dampen oscillations
    if epoch - last_improvement >= 15 && optimiser.eta > 1e-06 #1e-06
        optimiser.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(optimiser.eta)!")
        last_improvement = epoch
    end
    
    if epoch - last_improvement >= 25
        @warn(" -> We're calling this converged.")
        break
    end
end

96.1566225


┌ Info: [1]: Test accuracy: 0.6944
└ @ Main In[6]:14
┌ Info:  -> New best accuracy! Saving model out to fmnist_flux.bson
└ @ Main In[6]:23


183.6960653


┌ Info: [2]: Test accuracy: 0.7135
└ @ Main In[6]:14
┌ Info:  -> New best accuracy! Saving model out to fmnist_flux.bson
└ @ Main In[6]:23


270.4484071


...

┌ Info: [98]: Test accuracy: 0.7277
└ @ Main In[6]:14


11702.651708896


┌ Info: [99]: Test accuracy: 0.7277
└ @ Main In[6]:14


11789.689309796


┌ Info: [100]: Test accuracy: 0.7277
└ @ Main In[6]:14


In [7]:
time /= 1.0e9
print(time)

11789.689309796